In [1]:
from fastai.vision import *
from fastai import *
from ipywidgets import *
from gradcam import *
import torchvision.transforms.functional as F
from matplotlib import cm
%matplotlib inline

# DeepFake-Buster !

Show me a picture of a face that you think might be fake and I'll tell you what I think of it !

In [2]:
path = Path()

In [3]:
learn_inf = load_learner(path,'app.pkl')

In [4]:
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Show Grad-CAM')
out_pl = widgets.Output()
out_grad = widgets.Output()
lbl_pred = widgets.Label()

In [5]:
def on_click(change):
    img = open_image(io.BytesIO(btn_upload.data[-1]))
    out_pl.clear_output(wait=True)
    with out_pl: display(img.resize((3,256,256)))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'I\'m {100*probs[pred_idx]:.02f}% sure that this image is {pred} !'

In [6]:
def on_click_grad_cam(change):
    img = open_image(io.BytesIO(btn_upload.data[-1]))
    target_layer = learn_inf.model._bn1
    gcam = GradCam.from_one_img(learn_inf,target_layer, img)
    out_pl.clear_output()
    blended = get_hm(img, gcam.hmap1)
    with out_pl: display(blended.resize((256,256)))
        
btn_run.on_click(on_click_grad_cam)

In [7]:
def get_hm(img, hm):
    bg = F.to_pil_image(img.data)

    hm = (hm-hm.min())/(hm.max()-hm.min())

    fg = PIL.Image.fromarray(np.uint8(cm.magma(hm)*255))

    fg = fg.resize(bg.size, PIL.Image.BILINEAR)

    bg = bg.convert('RGBA')

    blended = PIL.Image.blend(bg, fg, alpha=0.5)
    return blended
    

In [8]:
btn_upload.observe(on_click, names=['data'])

In [9]:
display(VBox([widgets.Label('Select your image!'), btn_upload, btn_run,lbl_pred, out_pl]))